In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import Dataset

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
with open('sample_dataset.txt', 'r', encoding='utf-8') as file:
    text = file.read()
paragraphs = text.split('\n\n')

dataset = Dataset.from_dict({"text": paragraphs})

def tokenize_function(examples):
    inputs = tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

model = GPT2LMHeadModel.from_pretrained('gpt2')

Map:   0%|          | 0/6 [00:00<?, ? examples/s]

In [ ]:
import pandas as pd

training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10,
    save_total_limit=2,
    logging_dir='./logs',
)

df = tokenized_datasets.to_pandas()
processed_dataset = Dataset.from_pandas(df)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
)
trainer.train()

Step,Training Loss


TrainOutput(global_step=9, training_loss=2.2813186645507812, metrics={'train_runtime': 335.5779, 'train_samples_per_second': 0.054, 'train_steps_per_second': 0.027, 'total_flos': 4703256576000.0, 'train_loss': 2.2813186645507812, 'epoch': 3.0})

In [ ]:
model.save_pretrained('fine_tuned_gpt2')
tokenizer.save_pretrained('fine_tuned_gpt2')

input_text = "In the bustling city"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
attention_mask = torch.ones(input_ids.shape, device=input_ids.device)

sample_output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=150,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)

In [ ]:
input_text = "In the bustling city"
input_ids = tokenizer.encode(input_text, return_tensors='pt')
attention_mask = torch.ones(input_ids.shape, device=input_ids.device)

sample_output = model.generate(
    input_ids,
    attention_mask=attention_mask,
    max_length=150,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)
generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)
print(generated_text)

In the bustling city of the city, there are many people who are waiting for the arrival of their Lord.
